In [1]:
import collections
import matplotlib.pyplot as plt
import math
import nltk
import os

import pandas as pd
import pickle
import psycopg2
import re
import string
import tensorflow as tf


from utils_prep_comment_for_training import prepCommentForTraining 

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)

from credentials import credentials

In [2]:
from utils_fit_and_predict import fit_make_crosstab, get_model
from utils_env import get_working_dir

In [3]:
get_working_dir('model')

In [4]:
# Enter the twitter oauth details:
conn = psycopg2.connect(
        host=credentials['host'],
        port = credentials['port'],
        database=credentials['database'],
        user=credentials['user'],
        password=credentials['password'],
)
# Create a cursor object
    
cursor = conn.cursor()

## Identify principles 1-5*

#### Import the binary trainingData


In [5]:
SQLCommand = ("""

    SELECT source, 
           commentID,
           cleanedComment,
           recommend,
           CASE
              WHEN fastAccess >= 1 THEN 1
              ELSE 0
           END AS fastAccess,
           CASE
              WHEN effTreat >= 1 THEN 1
              ELSE 0
           END AS effTreat,
           CASE
              WHEN emotSup >= 1 THEN 1
              ELSE 0
           END AS emotSup,
           CASE
              WHEN apen >= 1 THEN 1
              ELSE 0
           END AS apen
      FROM trainingData
     WHERE fastAccess IS NOT NULL
       AND effTreat IS NOT NULL
       AND emotSup IS NOT NULL
       AND apen IS NOT NULL
       AND source != 'miscTweets'
       AND cleanedComment IS NOT NULL
       AND cleanedComment != ''

""")

cursor.execute(SQLCommand)
df = pd.DataFrame(cursor.fetchall())
df.columns = ['source','commentID','cleanedComment','recommend','fastAccess','effTreat','emotSup','apen']

df = df[~((df['source'] == 'twitter') & (df['recommend'] == 0))].reset_index(drop=True)

### Prep the comments for modelling and train the fastAccess model

In [6]:
prepped_data = prepCommentForTraining(df, 'cleanedComment', 'comment', 'fastAccess', 
                                      'binary', 2540, 5000, 0.8, 'None', 'PRINCIPLES_PROB')
Params = collections.namedtuple('Params',["vocab_size", "embed_size", "n_filters", 
                              "n_words", "batch_size", "n_epochs"])
params = Params(5000, 100, 128, 3, 256, 1)
model = get_model(prepped_data.maxlen, params)
domain = 'fastAccess'
probY_threshold = 0.4

# Above params reduced for speed. Original values:
# params = Params(5000, 100, 256, 3, 256, 50)

fit_make_crosstab(prepped_data, model, params, domain, probY_threshold);

testing
case set to lower
+ replaced with &
forward slashes replaced with space
u200ef etc removed
a & e correct to a&e
a&e changed to ae to deal with tokenizing nightmare!
non-standard characters removed
2 newly empty/blank comments removed from trainingData
trainProportion = math.ceil(len(df)*training_proportion)
Generating new word2index model

maxlen in training data: 2540
maxlen set to: 2540
word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
                    word2index[word[0]] = wid + 1
                #vocab_sz = len(word2index) + 1
                index2word = {v:k for k, v in word2index.items()}
word2index_Name: 20210105 trainingData PRINCIPLES_PROB word2index maxLen 2540.pickle

word2index pickle dumped, name: 20210105 trainingData PRINCIPLES_PROB word2index maxLen 2540.pickle
index2word_Name: 20210105 trainingData index2word maxLen 2540.pickle
X number sorted
Y number sorted
20210105fastAccess ALL PRINCIPLE_PROB CNN v500

C:\Users\Joe\PEP Health\Tech - Tech\Scripts\Python\run-model\utils_fit_and_predict.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['predicted'][results['probY'] >= probY_threshold] = 'y'
C:\Users\Joe\PEP Health\Tech - Tech\Scripts\Python\run-model\utils_fit_and_predict.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['actual'][results['actual'] == 0] = 'n'
